In [1]:
import pandas as pd
import random
import torch
import numpy as np
import re
import nltk
from transformers import GPT2LMHeadModel
from inference import get_tokenizer,get_masks,infill_with_ilm,infill_with_naive,get_additional_id
from helper import get_color_fonts,apply_masked_spans_with_color,remove_color_fonts

additional_tokens_to_ids = get_additional_id('../ILM/additional_ids_to_tokens.pkl')
tokenizer = get_tokenizer('gpt2',additional_tokens_to_ids)
true_color = get_color_fonts(colors='BackgroundCyan', tokenizer = tokenizer)
fake_color = get_color_fonts(colors='BackgroundRed', tokenizer = tokenizer)

additional_tokens to be addded to GPT2 tokenizer: 
{'<|startofinfill|>': 50257, '<|endofinfill|>': 50258, '<|infill_document|>': 50259, '<|infill_paragraph|>': 50260, '<|infill_sentence|>': 50261, '<|infill_ngram|>': 50262, '<|infill_word|>': 50263}


# Read Data

In [2]:
with open('../data/raw_data/cs_clean/test.txt', 'r') as f:
    raw = f.read().split('\n\n\n')
print(len(raw))

df = pd.DataFrame(raw, columns = ['doc'])
# df['words'] =  df['doc'].apply(lambda x: len(tokenizer.encode(x)))
# df1 = df[((df.words>180) & (df.words<200))] 
# Idx = sorted(df1.sample(n=5).index)

Idx = [3005,3206]


original_doc_list = df.doc[Idx]
masked_data = get_masks(original_doc_list)

3482




Original text
A Framework for Understanding Unintended Consequences of Machine Learning
As machine learning increasingly affects people and society, it is important that we strive for a comprehensive and unified understanding of how and why unwanted consequences arise. For instance, downstream harms to particular groups are often blamed on "biased data," but this concept encompass too many issues to be useful in developing solutions. In this paper, we provide a framework that partitions sources of downstream harm in machine learning into five distinct categories spanning the data generation and machine learning pipeline. We describe how these issues arise, how they are relevant to particular applications, and how they motivate different solutions. In doing so, we aim to facilitate the development of solutions that stem from an understanding of application-specific populations and data generation processes, rather than relying on general claims about what may or may not be "fai

In [3]:
for doc_id, (real_id, (doc, examples)) in enumerate(zip(Idx, masked_data)):
    if len(examples) == 0:
        continue
    masked_spans = random.choice(examples)
    mask_span_type_to_str = {t: '<|infill_{}|>'.format(str(t).split(".")[1].lower()) for t, _, _ in masked_spans}
    context, answers, color_text = apply_masked_spans_with_color(
        doc,
        masked_spans,
        mask_span_type_to_str, true_color)
    
    print('-' * 80)
    print('-' * 40,doc_id, real_id,  '-'* 40 )
    print('-' * 80)
    print(' ' * 36 + 'ORIGINAL')
    print(color_text)

--------------------------------------------------------------------------------
---------------------------------------- 0 3005 ----------------------------------------
--------------------------------------------------------------------------------
                                    ORIGINAL
A Framework for Understanding Unintended Consequences of Machine Learning
As machine learning increasingly affects people and society, it is important that we strive for a comprehensive and unified understanding of how and why unwanted consequences arise. For instance, downstream harms to particular groups are often blamed on "biased data," but this concept encompass too many issues to be useful in developing solutions. In this paper, we provide a framework that partitions sources of downstream harm in machine learning into five distinct categories spanning the data generation and machine learning pipeline. We describe how these issues arise, how they are relevant to particular applications, and

# FDI

In [4]:
TASK = 'ilm'
DATASET = 'cs_clean'
MAX_SEQ_LEN  = 512

NUM_FAKE = 3
MODEL_DIR = '../model/%s_%s'%(DATASET, TASK)
print(MODEL_DIR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)
model.eval()
_ = model.to(device)


penalty = 1.2

for doc_id, (real_id, (doc, examples)) in enumerate(zip(Idx, masked_data)):
    if len(examples) == 0:
        continue
    masked_spans = random.choice(examples)
    mask_span_type_to_str = {t: '<|infill_{}|>'.format(str(t).split(".")[1].lower()) for t, _, _ in masked_spans}
    context, answers, color_text = apply_masked_spans_with_color(
        doc,
        masked_spans,
        mask_span_type_to_str, true_color)
    
    print('-' * 80)
    print('-' * 40,doc_id, real_id,  '-'* 40 )
    print('-' * 80)
    print(' ' * 36 + 'ORIGINAL')
    print(color_text)
        
    
#     context_ids = tokenizer.encode(context)
    context1  = context.replace('<|infill_ngram|>', '________').replace('<|infill_word|>', '________').replace('<|infill_sentence|>', '________')
    context_ids = tokenizer.encode(context1)
    _blank_id = tokenizer.encode('________')[0]
    for i in masked_spans:
        keys =  mask_span_type_to_str[i[0]]
        context_ids[context_ids.index(_blank_id)] = tokenizer.encode(keys)[0]
    
    generated = infill_with_ilm(
    model,
    tokenizer,
    additional_tokens_to_ids,
    context_ids,
    num_infills=NUM_FAKE,
    max_sequence_length=MAX_SEQ_LEN,
    temp=1,
    topk=None,
    nucleus=0.95,
    add_colors=fake_color,
    penalty=penalty,
    answers=answers)
    
    generated_samples = [] 
    
    for idx, g in enumerate(generated):
        print('-' * 80)
        print(' ' * 36 + 'FAKE ' + str(idx))
        fake_with_color = tokenizer.decode(g)
        print(fake_with_color)
        print("-" *40)
        fake_no_color = remove_color_fonts(fake_with_color,fake_color)
        print(fake_no_color)

../model/cs_clean_ilm
--------------------------------------------------------------------------------
---------------------------------------- 0 3005 ----------------------------------------
--------------------------------------------------------------------------------
                                    ORIGINAL
A Framework for Understanding Unintended Consequences of Machine Learning
As machine learning increasingly affects people and society, it is important that we strive for a comprehensive and unified understanding of how and why unwanted consequences arise. For instance, downstream harms to particular groups are often blamed on "biased data," but this concept encompass too many issues to be useful in developing solutions. In this paper, we provide a framework that partitions sources of downstream harm in machine learning into five distinct categories spanning the data generation and machine learning pipeline. We describe how these issues arise, how they are relevant to partic

# GPT-2

In [5]:
TASK = 'lm'
DATASET = 'cs_clean'
MAX_SEQ_LEN  = 512
NUM_FAKE = 3
MODEL_DIR = '../model/%s_%s'%(DATASET, TASK)
print(MODEL_DIR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)
model.eval()
_ = model.to(device)


for doc_id, (doc, examples) in enumerate(masked_data):
    if len(examples) == 0:
        continue
    
    print('-' * 80)
    print('-' * 40,doc_id, '-'* 40 )
    print('-' * 80)
    print(' ' * 36 + 'ORIGINAL')
    print(doc)
        
    prompt = doc.split('\n')[0]+ '\n'+ nltk.sent_tokenize(doc.split('\n')[1])[0]
    # prompt = doc.split('\n')[0] +'\n'
    context_ids = tokenizer.encode(prompt)

    generated = infill_with_naive(
    model,
    tokenizer,
    additional_tokens_to_ids,
    context_ids,
    num_infills=NUM_FAKE *2,
    max_sequence_length=MAX_SEQ_LEN,
    temp=1,
    topk=None,
    nucleus=0.95)
    generated_samples = [] 
    
    
    # GPT-2 can't control the lengths, so we can select the outputs with the lengths closest to the original stories.
    
    original_len = len(tokenizer.encode(doc))
    diff = np.array([abs(len(g)-original_len) for g in generated])
    smallest = np.argpartition(diff,NUM_FAKE)[:NUM_FAKE]
    
    np.random.shuffle(smallest)
    for idx, i in enumerate(smallest):
        g = generated[i]
        print('-' * 80)
        print(' ' * 36 + 'FAKE ' + str(idx))
        tmp = tokenizer.decode(g)
        tmp = prompt +  ' ' + tmp.strip()[:1].upper() + tmp.strip()[1:]
        print(tmp)

../model/cs_clean_lm
--------------------------------------------------------------------------------
---------------------------------------- 0 ----------------------------------------
--------------------------------------------------------------------------------
                                    ORIGINAL
A Framework for Understanding Unintended Consequences of Machine Learning
As machine learning increasingly affects people and society, it is important that we strive for a comprehensive and unified understanding of how and why unwanted consequences arise. For instance, downstream harms to particular groups are often blamed on "biased data," but this concept encompass too many issues to be useful in developing solutions. In this paper, we provide a framework that partitions sources of downstream harm in machine learning into five distinct categories spanning the data generation and machine learning pipeline. We describe how these issues arise, how they are relevant to particular a